In [1]:
# dict {'5': 27, '6': 11, '4': 4, '9': 17, '8': 30, '7': 31}
"""
    1. get data, label
    2. split to train/test
    3. train svmc
    4. test, show acc
"""
from setMysqlData import *
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import svm

rng = np.random.RandomState(0)

mydb = connectDatabase("smartBottle")
result = getAllData(mydb, "drinkDataEnhan")
x = list()
y = list()
for line in result:
    data = [float(num) for num in line[1].split(',')]
    label = [float(num) for num in line[2].split(',')]
    x.append(data)
    y.append(int(sum(label)/10))
X = np.array(x)
Y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=rng)
# (100, 70) (20, 70) (100,) (20,)
clf = svm.SVC()
clf.fit(X_train, y_train)
pre = clf.predict(X_test)
# print(pre)
# print(y_test)
count = sum(x == y for x, y in zip(pre, y_test))
print(f'acc {count / len(pre)}')



acc 0.5384615384615384


In [7]:
import os
from setMysqlData import *
import pandas as pd
mydb = connectDatabase("smartBottle")
mycursor = mydb.cursor()
deleSql = "delete from drinkData"
mycursor.execute(deleSql)
mydb.commit()
path = "./data/"
"""
需要筛选的只是标记了50ml但是S/E相同
"""
files = os.scandir(path)
for f in files:
    df = pd.read_excel(f.path)
    start, end = int(df.loc[0, 'S']), int(df.loc[0, 'E'])
    drink = f.path.split('/')[-1].split('_')[0]
    if drink not in ['Stable', 'Walk']:
        if start == end:
            continue
        drink = int(drink[:-2])
    else:
        drink = 0
    y = df.loc[:, 'Y']
    z = df.loc[:, 'Z']
    y = [str(round(num/1080, 3)) for num in y]
    z = [str(round(num/1080, 3)) for num in z]
    label = ['0' for i in range(len(y))]
    for i in range(start-1, end-1):
        label[i] = str(round(drink / (end - start), 3))
    data_y = ','.join(y)
    data_z = ','.join(z)
    label  = ','.join(label)
    sql = "insert into drinkData (user_id, data_y, data_z, label) VALUES(%s, %s, %s, %s)"
    val = (1, data_y, data_z, label)
    mycursor.execute(sql, val)
mydb.commit()



In [6]:
a = '100ml'
b = int(a[:-2])
c = b / 10
c

10.0